In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm
from genetic_selection import GeneticSelectionCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

In [2]:
df = pd.read_csv('../data/jak3_descriptors.csv')
df

,Molecule ChEMBL ID,Smiles,pChEMBL Value,Molecule,MaxEStateIndex,MinEStateIndex,MaxAbsEStateIndex,MinAbsEStateIndex,qed,MolWt,...,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea
0,CHEMBL385937,Cc1ccc(C(=O)Nc2cccc(C(F)(F)F)c2)cc1-c1ccc2nc(N...,7.70,<rdkit.Chem.rdchem.Mol object at 0x14ed56730>,12.917910,-4.483900,12.917910,0.071660,0.459524,422.410,...,0,0,0,0,0,0,0,0,0,0
1,CHEMBL1080111,O=c1[nH]c2cnc(-n3cnc4ccccc43)nc2n1C1CCOc2c(F)c...,7.52,<rdkit.Chem.rdchem.Mol object at 0x16667af80>,14.243005,-0.439546,14.243005,0.185077,0.490285,402.389,...,0,0,0,0,0,0,0,0,0,0
2,CHEMBL1081376,O=c1[nH]c2cnc(-n3cnc4ccccc43)nc2n1C1CCOc2c(Cl)...,6.77,<rdkit.Chem.rdchem.Mol object at 0x16667b220>,12.889483,-0.245588,12.889483,0.239832,0.473622,418.844,...,0,0,0,0,0,0,0,0,0,0
3,CHEMBL3642278,COc1ccc(Nc2nc3c(-c4cn[nH]c4)cccn3n2)cc1,6.29,<rdkit.Chem.rdchem.Mol object at 0x16667b290>,5.152457,0.533437,5.152457,0.533437,0.605781,306.329,...,0,0,0,0,0,0,0,0,0,0
4,CHEMBL3642348,CN1CCC(c2ccc(Nc3nc4c(Nc5ccccc5S(C)(=O)=O)cccn4...,6.85,<rdkit.Chem.rdchem.Mol object at 0x16667b300>,12.168844,-3.377002,12.168844,0.235711,0.425341,476.606,...,0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3266,CHEMBL5201705,C=CC(=O)Nc1cc(Nc2ncc(Cl)c(O[C@@H]3CO[C@H]4[C@@...,8.89,<rdkit.Chem.rdchem.Mol object at 0x1666d9460>,13.863638,-1.178358,13.863638,0.023255,0.457796,520.993,...,0,0,0,0,0,0,0,0,0,0
3267,CHEMBL5192736,C=CC(=O)Nc1cc(Nc2ncc(Cl)c(O[C@@H]3CO[C@H]4[C@@...,8.92,<rdkit.Chem.rdchem.Mol object at 0x1666d94d0>,13.884472,-3.073039,13.884472,0.006903,0.441622,538.983,...,0,0,0,0,0,0,0,0,0,0
3268,CHEMBL5194482,C=CC(=O)Nc1cccc(Cn2c(=O)ccc3cnc(Nc4ccc(N5CCN(C...,8.70,<rdkit.Chem.rdchem.Mol object at 0x1666d9540>,12.815756,-0.292129,12.815756,0.172028,0.379282,495.587,...,0,0,0,0,0,0,0,0,0,0
3269,CHEMBL5088804,COc1cc(-c2ccc(O)cc2)cc2cnc(N)nc12,7.40,<rdkit.Chem.rdchem.Mol object at 0x1666d95b0>,9.348054,0.215911,9.348054,0.215911,0.745476,267.288,...,0,0,0,0,0,0,0,0,0,0


In [3]:
def activity(s):
    if s >= 6.5:
        return 1
    else:
        return 0 

In [4]:
df['activity'] = df['pChEMBL Value'].apply(activity)

In [7]:
X = df.iloc[:,4:-1]
y = df.iloc[:,-1]

In [9]:
X

,MaxEStateIndex,MinEStateIndex,MaxAbsEStateIndex,MinAbsEStateIndex,qed,MolWt,HeavyAtomMolWt,ExactMolWt,NumValenceElectrons,NumRadicalElectrons,...,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea
0,12.917910,-4.483900,12.917910,0.071660,0.459524,422.410,405.274,422.135446,156,0,...,0,0,0,0,0,0,0,0,0,0
1,14.243005,-0.439546,14.243005,0.185077,0.490285,402.389,387.269,402.124052,148,0,...,0,0,0,0,0,0,0,0,0,0
2,12.889483,-0.245588,12.889483,0.239832,0.473622,418.844,403.724,418.094501,148,0,...,0,0,0,0,0,0,0,0,0,0
3,5.152457,0.533437,5.152457,0.533437,0.605781,306.329,292.217,306.122909,114,0,...,0,0,0,0,0,0,0,0,0,0
4,12.168844,-3.377002,12.168844,0.235711,0.425341,476.606,448.382,476.199445,176,0,...,0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3266,13.863638,-1.178358,13.863638,0.023255,0.457796,520.993,490.753,520.200109,194,0,...,0,0,0,0,0,0,0,0,0,0
3267,13.884472,-3.073039,13.884472,0.006903,0.441622,538.983,509.751,538.190688,200,0,...,0,0,0,0,0,0,0,0,0,0
3268,12.815756,-0.292129,12.815756,0.172028,0.379282,495.587,466.355,495.238273,188,0,...,0,0,0,0,0,0,0,0,0,0
3269,9.348054,0.215911,9.348054,0.215911,0.745476,267.288,254.184,267.100777,100,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
y.value_counts()

activity
1    2170
0    1101
Name: count, dtype: int64

remove correlated features

In [12]:
def remove_correlated_features(descriptors):
    # Calculate correlation
    correlated_matrix = descriptors.corr().abs()

    # Upper triangle of correlation matrix
    upper_triangle = correlated_matrix.where(np.triu(np.ones(correlated_matrix.shape),k=1).astype(bool))

    # Identify columns that have above 0.9 values of correlation
    to_drop = [column for column in upper_triangle.columns if any(upper_triangle[column] >= 0.9)]
    print(to_drop)
    descriptors_correlated_dropped = descriptors.drop(columns=to_drop, axis=1)
    return descriptors_correlated_dropped   
descriptors_new = remove_correlated_features(X)
descriptors_new

['MaxAbsEStateIndex', 'HeavyAtomMolWt', 'ExactMolWt', 'NumValenceElectrons', 'MaxAbsPartialCharge', 'MinAbsPartialCharge', 'FpDensityMorgan2', 'FpDensityMorgan3', 'Chi0', 'Chi0n', 'Chi0v', 'Chi1', 'Chi1n', 'Chi1v', 'Chi2n', 'Chi2v', 'Chi3n', 'Chi3v', 'Chi4n', 'Chi4v', 'Kappa1', 'Kappa2', 'Kappa3', 'LabuteASA', 'SlogP_VSA6', 'VSA_EState10', 'VSA_EState6', 'HeavyAtomCount', 'NumHDonors', 'NumSaturatedCarbocycles', 'MolMR', 'fr_COO', 'fr_COO2', 'fr_C_O_noCOO', 'fr_Nhpyrrole', 'fr_amide', 'fr_benzene', 'fr_nitrile', 'fr_phenol', 'fr_phenol_noOrthoHbond']


,MaxEStateIndex,MinEStateIndex,MinAbsEStateIndex,qed,MolWt,NumRadicalElectrons,MaxPartialCharge,MinPartialCharge,FpDensityMorgan1,BCUT2D_MWHI,...,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea
0,12.917910,-4.483900,0.071660,0.459524,422.410,0,0.415986,-0.367775,1.032258,19.413184,...,0,0,0,0,0,0,0,0,0,0
1,14.243005,-0.439546,0.185077,0.490285,402.389,0,0.327804,-0.490048,1.133333,19.142261,...,0,0,0,0,0,0,0,0,0,0
2,12.889483,-0.245588,0.239832,0.473622,418.844,0,0.327804,-0.491561,1.133333,35.495692,...,0,0,0,0,0,0,0,0,0,0
3,5.152457,0.533437,0.533437,0.605781,306.329,0,0.247033,-0.496766,1.173913,16.465279,...,0,0,0,0,0,0,0,0,0,0
4,12.168844,-3.377002,0.235711,0.425341,476.606,0,0.247089,-0.351377,0.911765,32.227747,...,0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3266,13.863638,-1.178358,0.023255,0.457796,520.993,0,0.247322,-0.467813,1.277778,35.495693,...,0,0,0,0,0,0,0,0,0,0
3267,13.884472,-3.073039,0.006903,0.441622,538.983,0,0.298693,-0.467811,1.324324,35.495693,...,0,0,0,0,0,0,0,0,0,0
3268,12.815756,-0.292129,0.172028,0.379282,495.587,0,0.251994,-0.368965,1.027027,16.148956,...,0,0,0,0,0,0,0,0,0,0
3269,9.348054,0.215911,0.215911,0.745476,267.288,0,0.220171,-0.507966,1.100000,16.466008,...,0,0,0,0,0,0,0,0,0,0


remove LOW VARRIANCE FEATURES

In [13]:
from sklearn.feature_selection import VarianceThreshold

def remove_low_variance(input_data, threshold=0.1):
    selection = VarianceThreshold(threshold)
    selection.fit(input_data)
    return input_data[input_data.columns[selection.get_support(indices=True)]]

X = remove_low_variance(descriptors_new, threshold=0.1)
X

,MaxEStateIndex,MinEStateIndex,MolWt,BCUT2D_MWHI,BCUT2D_MRHI,BertzCT,HallKierAlpha,Ipc,PEOE_VSA1,PEOE_VSA10,...,fr_ether,fr_halogen,fr_imidazole,fr_methoxy,fr_para_hydroxylation,fr_piperdine,fr_priamide,fr_pyridine,fr_sulfonamd,fr_unbrch_alkane
0,12.917910,-4.483900,422.410,19.413184,6.048968,1301.737541,-3.94,8.392157e+06,11.050456,0.000000,...,0,3,0,0,0,0,0,0,0,0
1,14.243005,-0.439546,402.389,19.142261,5.770843,1484.211000,-4.01,1.917869e+07,9.720841,11.844021,...,1,1,2,0,3,0,0,0,0,0
2,12.889483,-0.245588,418.844,35.495692,6.317418,1484.211000,-3.65,1.917869e+07,9.720841,17.593533,...,1,1,2,0,3,0,0,0,0,0
3,5.152457,0.533437,306.329,16.465279,5.768679,927.036207,-3.35,4.534070e+05,10.053652,5.749512,...,1,0,0,1,0,0,0,1,0,0
4,12.168844,-3.377002,476.606,32.227747,7.904663,1407.132376,-3.43,6.823449e+07,15.533487,0.000000,...,0,0,0,0,1,1,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3266,13.863638,-1.178358,520.993,35.495693,6.312585,1106.271574,-2.99,1.237595e+08,34.643986,17.230566,...,3,2,0,0,0,0,0,0,0,0
3267,13.884472,-3.073039,538.983,35.495693,6.312588,1154.217451,-3.06,1.614355e+08,34.643986,17.733482,...,3,3,0,0,0,0,0,0,0,0
3268,12.815756,-0.292129,495.587,16.148956,5.984079,1488.935355,-4.50,2.869697e+08,20.433397,5.647177,...,0,0,0,0,0,0,0,1,0,0
3269,9.348054,0.215911,267.288,16.466008,5.894574,769.920679,-2.82,4.747005e+04,15.577058,17.015724,...,1,0,0,1,0,0,0,0,0,0


NORMALIZE THE DATA USING MINMAX


In [14]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler



# Use MinMaxScaler to normalize the DataFrame
scaler = MinMaxScaler()
X = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)

In [15]:
X

,MaxEStateIndex,MinEStateIndex,MolWt,BCUT2D_MWHI,BCUT2D_MRHI,BertzCT,HallKierAlpha,Ipc,PEOE_VSA1,PEOE_VSA10,...,fr_ether,fr_halogen,fr_imidazole,fr_methoxy,fr_para_hydroxylation,fr_piperdine,fr_priamide,fr_pyridine,fr_sulfonamd,fr_unbrch_alkane
0,0.765211,0.352055,0.401598,0.040196,0.066227,0.470751,0.440079,1.127814e-05,0.239647,0.000000,...,0.000000,0.333333,0.0,0.00,0.000000,0.0,0.0,0.000000,0.0,0.0
1,0.882954,0.835580,0.367263,0.037777,0.034018,0.559473,0.426326,2.578217e-05,0.210812,0.284938,...,0.166667,0.111111,1.0,0.00,1.000000,0.0,0.0,0.000000,0.0,0.0
2,0.762685,0.858769,0.395482,0.183788,0.097317,0.559473,0.497053,2.578217e-05,0.210812,0.423257,...,0.166667,0.111111,1.0,0.00,1.000000,0.0,0.0,0.000000,0.0,0.0
3,0.075203,0.951905,0.202526,0.013876,0.033767,0.288565,0.555992,6.033672e-07,0.218030,0.138319,...,0.166667,0.000000,0.0,0.25,0.000000,0.0,0.0,0.333333,0.0,0.0
4,0.698652,0.484391,0.494540,0.154611,0.281136,0.521996,0.540275,9.174466e-05,0.336869,0.000000,...,0.000000,0.000000,0.0,0.00,0.333333,0.5,0.0,0.333333,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3266,0.849245,0.747251,0.570662,0.183788,0.096757,0.375712,0.626719,1.664059e-04,0.751312,0.414525,...,0.500000,0.222222,0.0,0.00,0.000000,0.0,0.0,0.000000,0.0,0.0
3267,0.851096,0.520731,0.601513,0.183788,0.096757,0.399024,0.612967,2.170667e-04,0.751312,0.426624,...,0.500000,0.333333,0.0,0.00,0.000000,0.0,0.0,0.000000,0.0,0.0
3268,0.756134,0.853204,0.527092,0.011052,0.058713,0.561770,0.330059,3.858652e-04,0.443132,0.135857,...,0.000000,0.000000,0.0,0.00,0.000000,0.0,0.0,0.333333,0.0,0.0
3269,0.448008,0.913943,0.135573,0.013883,0.048347,0.212173,0.660118,5.752734e-08,0.337814,0.409356,...,0.166667,0.000000,0.0,0.25,0.000000,0.0,0.0,0.000000,0.0,0.0


In [16]:
#estimator = DecisionTreeClassifier()
estimator = RandomForestClassifier()

model = GeneticSelectionCV(
    estimator, cv=5, verbose=0,
    scoring="accuracy", max_features=15,
    n_population=100, crossover_proba=0.5,
    mutation_proba=0.2, n_generations=50,
    crossover_independent_proba=0.5,
    mutation_independent_proba=0.04,
    tournament_size=3, n_gen_no_change=10,
    caching=True, n_jobs=2)

model = model.fit(X, y)
print('Features:', X.columns[model.support_])

Features: Index(['MinEStateIndex', 'BCUT2D_MRHI', 'PEOE_VSA12', 'PEOE_VSA6', 'SMR_VSA2',
       'SMR_VSA6', 'SlogP_VSA8', 'VSA_EState5', 'NumAliphaticCarbocycles',
       'NumAromaticCarbocycles', 'NumAromaticRings', 'fr_Ar_N', 'fr_Ar_NH',
       'fr_aniline', 'fr_imidazole'],
      dtype='object')


In [17]:
lene = ['MinEStateIndex', 'BCUT2D_MRHI', 'PEOE_VSA12', 'PEOE_VSA6', 'SMR_VSA2',
       'SMR_VSA6', 'SlogP_VSA8', 'VSA_EState5', 'NumAliphaticCarbocycles',
       'NumAromaticCarbocycles', 'NumAromaticRings', 'fr_Ar_N', 'fr_Ar_NH',
       'fr_aniline', 'fr_imidazole']
len(lene)

15

In [22]:
X[['MinEStateIndex', 'BCUT2D_MRHI', 'PEOE_VSA12', 'PEOE_VSA6', 'SMR_VSA2',
       'SMR_VSA6', 'SlogP_VSA8', 'VSA_EState5', 'NumAliphaticCarbocycles',
       'NumAromaticCarbocycles', 'NumAromaticRings', 'fr_Ar_N', 'fr_Ar_NH',
       'fr_aniline', 'fr_imidazole']].to_csv('../data/15_best_features.csv')